In [ ]:
import mysql

In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import sqlalchemy

In [ ]:
# create the engine
connection = mysql.connector.connect(host='ns3142407.ip-51-77-118.eu',
                             database='kaiseki_prices',
                             user='remote_user',
                             password='Password*8')
# check is connected
connection.is_connected()

In [ ]:
# read SQL from text file
with open('db_stmt.txt', 'r') as file:
    stmt = file.read().strip()
file.close()

In [ ]:
# get data function
def getLatestPrices():
    cursor = connection.cursor()
    cursor.execute(stmt)
    result = cursor.fetchall()
    # Create a DataFrame from the results: df
    df = pd.DataFrame(result)
    # Set Column names
    df.columns = [i[0] for i in cursor.description]
    df.set_index('timestamp', inplace=True)
    df = df.apply(pd.to_numeric, errors='ignore')
    return df

In [ ]:
# create trades table
trades = getLatestPrices()

In [ ]:
# add columns
trades = trades.drop_duplicates(subset='trace_id', keep='first')
trades = trades.sort_index(ascending=True)
trades['pnl'] = trades['RealisedReturn'] / 2
trades['open_spread'] = (trades['OpenPrice'] / trades['openMid'])-1
trades['open_spread'] = np.sign(trades['Quantity']) * trades['open_spread'] 
trades['close_spread'] = (trades['ClosingPrice'] / trades['closeMid'])-1
trades['close_spread'] = -np.sign(trades['Quantity']) * trades['close_spread'] 

trades['time'] = trades.index
trades['time'] = trades['time'].dt.floor("H")
trades_pivoted = trades.pivot_table(index='time', columns='symbol',values = 'Quantity',aggfunc=np.sum)
trades_pivoted = trades_pivoted.fillna(0)


In [ ]:
trades

In [ ]:
pnl_pivoted = trades.pivot_table(index='time', columns='symbol',values = 'pnl', aggfunc=np.sum)
pnl_pivoted = pnl_pivoted.fillna(0)

pnl_pivoted['total'] = (pnl_pivoted.sum(axis=1))/2
pnl_pivoted = pnl_pivoted[pnl_pivoted.index.dayofweek < 5]

In [ ]:
# fetch hourly mids from database
cursor = connection.cursor()
# cursor.execute("SELECT (bid + offer) / 2 as mid, timestamp, symbol FROM kaiseki_prices.shapshots_hourly sh, kaiseki_prices.symbols s "\
#                "WHERE sh.symbol_id = s.id")
cursor.execute("SELECT *  FROM kaiseki_prices.bars_hourly bh join kaiseki_prices.Symbols s on bh.symbol_id  = s.id")
rows = cursor.fetchall()
prices = pd.DataFrame(rows)
prices.columns = [i[0] for i in cursor.description]
prices.set_index('timestamp', inplace=True)

In [ ]:
# create datetime index 
today = pd.to_datetime('today')
dti = pd.date_range(start='2019-11-01', end=today, freq='H')
# create prices and returns
prices_pivoted = prices.pivot_table(index='timestamp', columns='symbol', values = 'close', aggfunc=np.sum)
prices_pivoted.index.names = ['time']
prices_df = prices_pivoted.loc[dti,:]
returns = prices_pivoted.pct_change()
returns = returns.loc[dti,:]

In [ ]:
def Dollarise(df_close):
        df_close_dollarised = df_close.copy(deep=True)
        for col in df_close.columns:
            if col[0:3] != "USD":
                column_name = col[3:] + col[0:3]
                df_close_dollarised = df_close_dollarised.drop(col, axis=1)
                df_close_dollarised[column_name] = 1 / df_close[col]
        return df_close_dollarised

In [ ]:
prices_df = Dollarise(prices_df)

In [ ]:
# create theoretical model returns
theo_model_results = (np.sign(trades_pivoted) * returns )/ 2
theo_model_results['total'] = (theo_model_results.sum(axis=1))
theo_model_results = theo_model_results[theo_model_results.index.dayofweek < 5]

In [ ]:
ff_theo = theo_model_results.between_time('08:00', '19:00')

In [ ]:
pnl_pivoted.tail(70).to_clipboard()

In [ ]:
ff_theo.tail(100).to_clipboard()

In [ ]:
trades.tail()

In [ ]:
trades.tail(130).to_clipboard()


In [ ]:
np.sign(trades_pivoted).tail(20)

In [ ]:
prices_df.tail()

In [ ]:
ff_theo.tail(100).to_clipboard()

In [ ]:
np.sign(trades_pivoted).tail()

In [ ]:
returns.shift(-1).tail()

In [ ]:
breakout_trades = trades.loc[trades['Strategy'] == 'BreakOutstrategy']
